In [16]:
!pip install pandas


## Data download process
The data is available from https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs. The Refseq_species.tsv contains all the codon frequencies of many species from NCBI. Firstly we try to download via the download link with wget.

In [15]:
import os
import wget
print(myfile)
data_folder = "./data"
filename = "codon_usage.tsv"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
filepath = os.path.join(data_folder,filename)
url = "https://hive.biochemistry.gwu.edu/dna.cgi?cmd=objFile&ids=586358&filename=Refseq_species.tsv&raw=1"
wget.download(url, filepath)
with open("data/codon_usage.tsv", "r") as f:
    print(f.readlines())

<Response [200]>
[]


However, the above approach did not work and returned an empty file. After examining the download process with the developers tool in Google Chrome, I realise it requires some headers for the GET request. I tried the below copying the headers in but it still did not work.

In [9]:
import requests
headers = {
"Host": "hive.biochemistry.gwu.edu",
"Connection": "keep-alive",
"sec-ch-ua": '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
"sec-ch-ua-mobile": "?0",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "same-origin",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-User": "?1",
"Sec-Fetch-Dest": "document",
"Referer": "https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9",

"Range": "bytes=783482-783482",
"If-Range": "1578517013:86024007"}
myfile = requests.get("https://hive.biochemistry.gwu.edu/dna.cgi?cmd=objFile&ids=586358&filename=Refseq_species.tsv&raw=1", headers=headers)
print(myfile.content)

Once again, it is empty. Hence, I resorted to using Selenium and navigating the browser. First, I downloaded the Chrome driver. Then, I used the browser developer tools to examine the webpage and copy the XPATH of the elements that have to be clicked. On normal navigation, it requires 2 clicks to initate the download of the file. 

In [ ]:
from zipfile import ZipFile
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

#downloading and extracting the chrome drive
wget.download("https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip")
file_name = "chromedriver_win32.zip"
with ZipFile(file_name, 'r') as zip:
    zip.extractall()

driver = webdriver.Chrome(executable_path=r'./chromedriver.exe')
driver.get("https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs")
timeout = 60
try:
    element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="results"]/li[3]/a'))
    WebDriverWait(driver, timeout).until(element_present)
    download_tab = driver.find_element_by_xpath("/html/body/form[2]/div/div/div[4]/div[1]/div[1]/div[2]/ul/li[3]/ul/li[3]/a")
    download_tab.click()
    print("Clicked the first button!")
except TimeoutException:
    print ("Timed out waiting for page to load")

try:
    element_present = EC.presence_of_element_located((By.XPATH, '/html/body/form[2]/div/div/div[7]/div[1]/div[2]/div/div/div[2]/table/tbody/tr[18]/td[2]'))
    WebDriverWait(driver, timeout).until(element_present)
    print("present!")
    download = driver.find_element_by_xpath('/html/body/form[2]/div/div/div[7]/div[1]/div[2]/div/div/div[2]/table/tbody/tr[18]/td[2]')
    download.click()
    print("Clicked the second tab!")
except TimeoutException:
    print ("Timed out waiting for page to load")

driver.close()

However, this method ultimately did not work as the second table did not appear after clicking the first click. I was unable to make more progress due to my limited knowledge of HTML and JavaScript. Ultimately, I posit that there are safeguards put in place to limit any automated downloading/conceal the download as they are an academic institute and may be afraid of request overload. I moved on by downloading the data manually as of 7th of March. 

## Data exploration
Here, we look at how many entries are there in the files. 

In [25]:
import pandas as pd
df = pd.read_table("data/o586358-Refseq_species.tsv",index_col=False,low_memory=False)
print(df.shape)
df.head()

(191025, 76)


,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
0,refseq,GCF_002287175.1,2161,Methanobacterium bryantii,genomic,11,3266,910953,35.52,45.52,...,2404,1515,12496,7687,14043,8418,18911,8417,26631,8789
1,refseq,GCF_000302455.1,1204725,Methanobacterium formicicum DSM 3637,genomic,11,2520,719599,40.88,49.69,...,2968,3068,10562,4882,8084,6214,17552,5296,21059,7069
2,refseq,GCF_000762265.1,2162,Methanobacterium formicicum,genomic,11,2376,678104,43.72,51.53,...,3067,5204,9335,4793,6499,5439,15945,5847,19130,7839
3,refseq,GCF_001458655.1,2162,Methanobacterium formicicum,genomic,11,2372,690178,43.50,51.30,...,3102,5182,9545,4797,6613,5472,16184,5887,19385,7934
4,refseq,GCF_000016525.1,420247,Methanobrevibacter smithii ATCC 35061,genomic,11,1757,557617,32.06,43.34,...,405,168,8234,3248,10191,2880,14664,2965,16981,2107


In [42]:
print(list(df))
print(df.iloc[:, 0:6].nunique())
df.iloc[:,4].unique()

['Division', 'Assembly', 'Taxid', 'Species', 'Organelle', 'Translation Table', '# CDS', '# Codons', 'GC%', 'GC1%', 'GC2%', 'GC3%', 'TTT', 'TTC', 'TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG', 'ATT', 'ATC', 'ATA', 'ATG', 'GTT', 'GTC', 'GTA', 'GTG', 'TAT', 'TAC', 'TAA', 'TAG', 'CAT', 'CAC', 'CAA', 'CAG', 'AAT', 'AAC', 'AAA', 'AAG', 'GAT', 'GAC', 'GAA', 'GAG', 'TCT', 'TCC', 'TCA', 'TCG', 'CCT', 'CCC', 'CCA', 'CCG', 'ACT', 'ACC', 'ACA', 'ACG', 'GCT', 'GCC', 'GCA', 'GCG', 'TGT', 'TGC', 'TGA', 'TGG', 'CGT', 'CGC', 'CGA', 'CGG', 'AGT', 'AGC', 'AGA', 'AGG', 'GGT', 'GGC', 'GGA', 'GGG']
Division                  1
Assembly             177439
Taxid                 59670
Species               59060
Organelle                 4
Translation Table        11
dtype: int64


array(['genomic', 'mitochondrion', 'chloroplast', 'plastid'], dtype=object)

As we can see here, we have 191025 samples with 76 variables. A quick glance at the number of unique values show that the data requires cleaning. Firstly, we have less unique assemblies than number of rows, meaning that some of the assemblies are repeated or NaN. Even though the number of species and taxid differs, here it is acceptable as there might taxonomic IDs only goes up till species levels but there might be subspecies especially for microbes. We also have different type of DNA origins, here we focus on genomic DNA as those genes in mitochondrion, plastid or chloroplast might be shared between distantly related species and skew the differences. 

In [45]:
df[df.duplicated(['Assembly'])]

,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
658,refseq,GCF_000513435.1,1173487,Halorubrum sp. AJ67,genomic,11,3078,763055,64.68,67.31,...,8366,17661,3223,10213,1988,2032,7707,29947,8715,16798
660,refseq,GCF_000513435.1,1173487,Halorubrum sp. AJ67,genomic,11,3083,764041,64.69,67.32,...,8369,17683,3223,10222,1988,2032,7711,30002,8725,16820
719,refseq,NaN,1407055,Candidatus Nitrosotenuis uzonensis,genomic,11,1,200,55.83,60.50,...,1,0,1,1,2,2,0,2,5,0
720,refseq,GCF_000723185.1,1407055,Candidatus Nitrosotenuis uzonensis,genomic,11,1876,493394,42.86,49.86,...,2243,824,3560,5730,9304,5257,6294,10132,12660,3397
4956,refseq,NaN,1354719,Helicobacter pylori HP87P7,genomic,11,4,1064,37.53,40.04,...,11,8,10,22,21,21,9,10,14,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191020,refseq,NaN,1923763,Xingshan nematode virus 4,genomic,0,5,3630,38.21,43.91,...,49,13,60,29,72,21,40,19,85,41
191021,refseq,NaN,1923772,Xinzhou nematode virus 4,genomic,0,5,3652,39.91,45.29,...,39,11,71,30,73,12,38,21,99,34
191022,refseq,NaN,1415147,Xylella phage Salvo,genomic,11,72,17848,63.15,62.21,...,58,140,30,344,10,22,163,993,71,109
191023,refseq,NaN,1415148,Xylella phage Sano,genomic,11,77,17851,62.46,61.59,...,71,137,41,340,10,24,185,993,74,95
